In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
x = tf.Variable(1.0, dtype=tf.float32)

In [7]:
@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return(x)


add_print(tf.constant(2.))

9


<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [9]:
class SelfModule(tf.Module):
    def __init__(self, init_value=tf.constant(0.0), name=None):
        super(SelfModule, self).__init__(name=name)
        with self.name_scope:
            self.x = tf.Variable(init_value, dtype=tf.float32, trainable=True)
            
    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
    def add_print(self, a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return (self.x)
        
        

In [11]:
sm = SelfModule()
sm.add_print(tf.constant(1.0))

1


<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [13]:
display(sm.variables, sm.trainable_variables)

(<tf.Variable 'self_module/Variable:0' shape=() dtype=float32, numpy=1.0>,)

(<tf.Variable 'self_module/Variable:0' shape=() dtype=float32, numpy=1.0>,)

In [14]:
sm.submodules

()

In [15]:
tf.saved_model.save(sm, './models/model_sm_1', signatures={'serving_default':sm.add_print})

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./models/model_sm_1/assets


In [17]:
sm = tf.saved_model.load('./models/model_sm_1')
sm.add_print(tf.constant(1.))

2


<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [18]:
!saved_model_cli show --dir ./models/model_sm_1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['a'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: serving_default_a:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: 'add_print'
    Option #1
      Callable wi

In [20]:
import datetime

In [23]:
# 创建日志
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = './models/demomodule/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

tf.summary.trace_on(graph=True, profiler=True)

In [24]:
sm = SelfModule(init_value=tf.constant(0.))
result = sm.add_print(tf.constant(5.))

with writer.as_default():
    tf.summary.trace_export(name='demomodule', step=0, profiler_outdir=logdir)

5


In [25]:
%reload_ext tensorboard

In [28]:
from tensorboard import notebook
notebook.list()

No known TensorBoard instances running.


In [29]:
notebook.start('--logdir ./models/demomodule/')

In [31]:
mymodule = tf.Module()
mymodule.x = tf.Variable(0.)

In [37]:
@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
def add_print(a):
    mymodule.x.assign_add(a)
    tf.print(mymodule.x)
    return (mymodule.x)

In [38]:
add_print(tf.constant(1.))

11


<tf.Tensor: shape=(), dtype=float32, numpy=11.0>

In [39]:
print(mymodule.variables)

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=11.0>,)


In [40]:
mymodule.trainable_variables

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=11.0>,)

In [41]:
mymodule.submodules

()

In [42]:
from tensorflow.keras import models, layers, losses, metrics

In [46]:
issubclass(tf.keras.Model, tf.Module)

True

In [49]:
issubclass(tf.keras.layers.Layer, tf.Module)

True

In [50]:
issubclass(tf.keras.Model, tf.keras.layers.Layer)

True

In [51]:
tf.keras.backend.clear_session()

In [52]:
model_1 = models.Sequential()

In [53]:
model_1.add(layers.Dense(4, input_shape=(10, )))
model_1.add(layers.Dense(2))
model_1.add(layers.Dense(1))

model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [54]:
model_1.variables

[<tf.Variable 'dense/kernel:0' shape=(10, 4) dtype=float32, numpy=
 array([[ 0.6090604 ,  0.6170784 , -0.47469127,  0.57209134],
        [-0.39442864, -0.10699737,  0.12390149,  0.4188671 ],
        [ 0.45134878,  0.27571738,  0.0809018 ,  0.64637184],
        [ 0.09126073, -0.62824106, -0.20729461, -0.22724324],
        [ 0.6082704 , -0.4902019 ,  0.5286126 , -0.5470109 ],
        [ 0.1042825 ,  0.26741493, -0.42957324,  0.55470884],
        [ 0.02195132,  0.27423036,  0.37573302,  0.39576757],
        [-0.08128852, -0.3656864 , -0.56055754, -0.35069272],
        [ 0.4039061 , -0.16385135,  0.01848364,  0.4841392 ],
        [-0.12005281,  0.25069702, -0.24762923,  0.448025  ]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[ 0.79527617, -0.8327987 ],
        [-0.19546533,  0.8051081 ],
        [ 0.8458276 , -0.04643917],
        [-

In [55]:
model_1.submodules

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x63cbf64e0>,
 <tensorflow.python.keras.layers.core.Dense at 0x63cbf6da0>)

In [56]:
model_1.layers

In [57]:
model_1.name

'sequential'

In [59]:
model_1.name_scope

<bound method Layer._name_scope of <tensorflow.python.keras.engine.sequential.Sequential object at 0x63dc68550>>